In [ ]:
#!pip install --upgrade openai

%reset -f

import json
import os
import openai

client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"), organization = os.getenv("OPENAI_ORGANIZATION"))

In [ ]:
# Here we set the role of the assistant.  Important to provide the role and format of the output.  In the public repo, I can not put any 
# any examples in the prompt.  However, I do think adding specific examples in the prompt here will really help performance (few shot learning).
# Also I put a flag in the role to put UNKNOWN in any field that it is unsure about.  This should be an easy way to search in the resulting
# excel file for any issues.

RoleInstructions = """I'm trying to parse text extracted from a form.  I'd like you to give me an output that looks like the following: \n
    
    Filename,    
    Type of Host, 
    Dog Host ID,
    Name, 
    Country,
    Province,
    District,
    Zone,
    Village,
    Emergence,
    # of Sample,
    Age,
    Sex,

    There will be text that needs to be removed.  It will look like the first line with the filename, but it could be intermingled in the 
    other text and be slightly misspelled. Please remove that. If there is a missing entry or field, put UNKNOWN.  Also, the Age field may
    have more than just a number. Whatever text is available for Age, put that in too.  The Zone and Village name may also have a number 
    in the name, include that in the text.  
    
    Each user query will be the text to parse.  

    User:  filename:PDB23-00002
    Type of Host: Dog Host ID: 2022A Name: Josh Country: USA Province: Washington District: King Zone: North Seattle village: Greenwood Emergence: 1/11/2020 # Sample: 1 Age: 45 Sex: M

    PDB23-000@2

    Assistant:  

    Filename, PDB23-00002
    Type of Host, Dog
    Host ID, 2022A
    Name, Josh
    Country, USA
    Province, Washington
    District, King
    Zone, North Seattle
    Village, Greenwood
    Emergence, 1/11/2020
    # of Sample, 1
    Age, 45
    Sex, M 
    """

#Create an assistant and set the underlying model

assistant = client.beta.assistants.create(
    name="GW Sample Form Reader",
    instructions=RoleInstructions,
    model="gpt-4-1106-preview",
)

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

#show_json(assistant)

In [ ]:
## Create a new thread This will hold the state of our conversation.  No need to send the whole message each time!

thread = client.beta.threads.create()
show_json(thread)

In [ ]:
## Create a run.  A run has a thread and assistant.

run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)
show_json(run)

In [ ]:
## Streaming not yet available

import time

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

In [ ]:
run = wait_on_run(run, thread)
show_json(run)

In [ ]:
# Create a message to append to our thread
message = client.beta.threads.messages.create(
    thread_id=thread.id, role="user", content="Country: Chad"
)

# Execute our run
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

# Wait for completion
wait_on_run(run, thread)

# Retrieve all the messages added after our last user message
messages = client.beta.threads.messages.list(
    thread_id=thread.id, order="asc", after=message.id
)
show_json(messages)

In [ ]:
Form_ASSISTANT_ID = assistant.id  # or a hard-coded ID like "asst-..."

def submit_message(assistant_id, thread, user_message):
    client.beta.threads.messages.create(
        thread_id=thread.id, role="user", content=user_message
    )
    return client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )

def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id, order="asc")

In [ ]:
def create_thread_and_run(user_input):
    thread = client.beta.threads.create()
    run = submit_message(Form_ASSISTANT_ID, thread, user_input)
    return thread, run

In [ ]:
def pretty_print(messages):
    print("# Messages")
    for m in messages:
        print(f"{m.role}: {m.content[0].text.value}")
    print()

In [ ]:
#Prepare the output of the LLM to be parsed into a dictionary to be saved as an excel file

# Define the specific keys you're interested in
keys_of_interest = ["Filename", "Type of Host","Host ID","Name","Country","Province","District","Zone","Village","Emergence","# of Sample","Age", "Sex"]

# Initialize an empty dictionary to store the 
if 'data_dict' in locals():
    data_dict.clear()
data_dict = {key: None for key in keys_of_interest}  # Pre-fill the dictionary with the keys and set their values to None

def FillDictionary(forms_list, data_string):
    # Create a new dictionary for the current form
    form_dict = {}
    lines = data_string.split('\n')  # Split the form into lines
    
    for line in lines:
        parts = line.split(', ', 1)  # Attempt to split each line
        if len(parts) == 2:
            key, value = parts
            form_dict[key] = value.strip()  # Add key-value pair to the form dictionary
        else:
            print(f"Warning: Line does not contain expected format: '{line}'")
    
    # Add the current form's dictionary to the list of forms
    forms_list.append(form_dict)


In [ ]:
#  This is the cell that actually calls the LLM.  All of the previous cells are setting up this cell.

import os
import pandas as pd

localFolderPathText = './textFiles'

# finds the txt file names in the local folder textFiles
files = os.listdir(localFolderPathText)
txt_files = [file for file in files if file.endswith('.txt')]  

# Here I have added a loop to produce multiple output files to test how repeatable the process is.

for index in range(1,3): 
    
    # Initialize an empty dictionary to store the 
    if 'data_dict' not in locals() or data_dict is None:
        data_dict = {}
    else:
        data_dict.clear()
        
    data_dict = {key: None for key in keys_of_interest}  # Pre-fill the dictionary with the keys and set their values to None
    
    forms_list = []  # Initialize an empty list to hold all form dictionaries

    for fil in txt_files:
        
        with open(localFolderPathText+'/'+fil,'r',encoding='utf-8') as file:
            prompt = file.read()

        # This runs the LLM model with the data from the text file
        thread, run = create_thread_and_run(prompt) 
        run = wait_on_run(run, thread) 
        run.status
    
        messages = get_response(thread)
    
        for m in messages: 
            assistantResponse = str(f"{m.content[0].text.value}")
    
        data_dict = FillDictionary(forms_list,assistantResponse)

    df = pd.DataFrame(forms_list)  # Convert the list of dictionaries to a pandas DataFrame
    
    # Specify your Excel file name
    excel_filename = 'forms_data_'+str(index)+'.xlsx'
    
    # Export the DataFrame to an Excel file
    df.to_excel(excel_filename, index=False, engine='openpyxl')
    
    print(f'Data saved to {excel_filename}')

In [ ]:
# This is a function to try and compare the multiple output files to assess reliability.

import pandas as pd

def compare_excel_files(file1, file2):
    # Load the Excel files
    xls1 = pd.ExcelFile(file1)
    xls2 = pd.ExcelFile(file2)

    # Assuming you want to compare the first sheet; you can adjust this as needed
    df1 = pd.read_excel(xls1, sheet_name=0)
    df2 = pd.read_excel(xls2, sheet_name=0)

    # Check if the shapes of the two dataframes are the same
    if df1.shape != df2.shape:
        print(f"The files have different shapes: {df1.shape} vs {df2.shape}")
        return

    # Compare the dataframes and count the differences
    comparison_values = df1.values == df2.values
    rows, cols = comparison_values.shape

    # Count how many differences
    differences = rows * cols - comparison_values.sum()
    
    if differences == 0:
        print("The files are exactly the same.")
    else:
        print(f"The files differ in {differences} entries.")
    return differences

# Specify your Excel file name

excel_filename = 'forms_data_1.xlsx'

for index in range(2,3):
# Replace 'file_path_1.xlsx' and 'file_path_2.xlsx' with the paths to your actual files
    compare_excel_files('forms_data_1.xlsx','forms_data_'+str(index)+'.xlsx')
